Analysis of WikiSRAT data
===

In this notebook, we analyze data in the WikiSRAT database that was fetched in WikiSRAT_Analysis.ipynb.

No plotting or visualizations are included in this notebook. 

Run within a conda environment specified in the included `environment.yml` file. 

Create the environment with either the Import button on Anaconda Navigator's Environments tab, or this Conda command in your terminal or console, replacing `path/environment.yml` with the full file pathway to the environment.yml file in the local cloned repository.

```bash
conda env create --file path/environment.yml
```

To update your environment, either use Anaconda Navigator, or run the following command:

```bash
conda env update --file path/environment.yml --prune
```

or

```bash
conda env create --file path/environment.yml --force
```

# Setup
* Import packages
* Load data from parquet files

In [3]:
# packages for data requests
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
import json
import os
from pathlib import Path
import numpy as np

# packages for viz 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from  matplotlib.colors import LogNorm
import warnings

# geo packages
import geopandas as gpd
from shapely.geometry import Polygon
import contextily as ctx

# custom plotting functions
from PlottingFunctions import *


ModuleNotFoundError: No module named 'PlottingFunctions'

In [2]:
print("Geopandas: ", gpd.__version__)
# print("spatialpandas: ", spd.__version__)
# print("datashader: ", ds.__version__)
# print("pygeos: ", pygeos.__version__)

Geopandas:  0.9.0


# Process Data for Pollution Assessment

## Units
* Concentration: mg/L
* loadrate_totals: kg/yr
* loadrate_total_ws: attempt to get loadrate totals to kg/ha: fails if mean annual flow doesn't exist = ((loadate_conc * 28.3168 * 31557600 / 1000000) * maflowv) / watershed_hectar

## Task 1. Threshold values for acceptable water quality

**Task 1. Establish credible external thresholds for the three focal “nonpoint source” pollutants**
- ... (phosphorus, nitrogen, and sediment) from among those already recommended by the EPA or other reputable researchers to define “healthy” water vis-à-vis these pollutants (please see disclaimer, below).
- We will review the scientific literature, state and federal stream water quality criteria, information from several TMDL studies, and other materials to arrive at recommendations for NPS annual load or average concentrations that define acceptable values or ranges. 

Task 1 is a pre-requisite for **DRWI Pollution Assement Goal 1**:
- Identify hotspots of excess nonpoint source pollution (nitrogen, phosphorus, sediment) in stream reaches and catchments of the Delaware River Watershed.

**Background**:
- Developed from Sheeder and Evans 2004, but not using the midpoint values they highlighted.
  - (i.e. 95% confidence that the watershed is not "Impaired")
- Details in [Pollutant Thresholds from TMDLs](https://docs.google.com/spreadsheets/d/1k2QHjVeXarQuRWMfbAMi6XABWP1o0cBmaLx4gj9iaKc/edit#gid=0) Google Doc.
  - Note that TN includes organic N, presumed to be 1/3 of the total.

```
    | Target TN Load Rate (kg/ha) | Target TP Load Rate (kg/ha) | Target Sed Load Rate (kg/ha)  
    | 17.07 | 0.31 | 923.80 
    
    | Target TN conc (mg/l) | Target TP conc (mg/l) | Target Sed conc (mg/l)  
    | 4.73 | 0.09 | 237.30 | 
 ```

In [7]:
# Threshold/Target Values for Acceptable Water Quality

# Catchment Target Load Rate (kg/ha)
tn_loadrate_target  = 17.07  # Includes Organic N
tp_loadrate_target  = 0.31
tss_loadrate_target = 923.80

# Reach Target Concenctration (mg/l)
tn_conc_target  = 4.73  # Includes Organic N
tp_conc_target  = 0.09
tss_conc_target = 237.30

# Minimum Values, to avoid negative numbers and errors with LOG normalized plots
# = Targets / 100

In [8]:
# Create a dictionary of these Targets, to use later for iterating functions

targets = {'tn':  {'loadrate_target':tn_loadrate_target,
                   'conc_target': tn_conc_target},
           'tp':  {'loadrate_target':tp_loadrate_target,
                   'conc_target': tp_conc_target},
           'tss': {'loadrate_target':tss_loadrate_target,
                   'conc_target': tss_conc_target}
          }

In [9]:
# demonstration of how to access information within the dictionary
print(targets['tp'])
print(targets['tp']['conc_target'])

{'loadrate_target': 0.31, 'conc_target': 0.09}
0.09


In [ ]:
# Check CRS, which appears preserved in Parquet file metadata.
base_catch_gdf.crs

<Projected CRS: EPSG:32618>
Name: WGS 84 / UTM zone 18N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 78°W and 72°W, northern hemisphere between equator and 84°N, onshore and offshore. Bahamas. Canada - Nunavut; Ontario; Quebec. Colombia. Cuba. Ecuador. Greenland. Haiti. Jamica. Panama. Turks and Caicos Islands. United States (USA). Venezuela.
- bounds: (-78.0, 0.0, -72.0, 84.0)
Coordinate Operation:
- name: UTM zone 18N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
%%time
# Test method to reproject CRS to 3857, which is useful for visualization
base_catch_gdf.to_crs(epsg=3857, inplace=True)

Wall time: 1.71 s


In [ ]:
# Check CRS
base_catch_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
%%time
# Automate method to reproject CRS to 3857, for all GeoDataFrames
gdfs = [base_catch_gdf, base_reach_gdf, rest_catch_gdf, rest_reach_gdf, point_src_gdf, proj_prot_gdf, proj_rest_gdf, cluster_gdf, focusarea_gdf]

for gdf in gdfs:
    gdf.to_crs(epsg=3857, inplace=True)

Wall time: 3.84 s


## Compare to MMW
Compare with Model My Watershed sub-basin modeling results for: 
- **West Branch Brandywine Creek, HUC-10**, available at https://modelmywatershed.org/project/36183/.
  - **Upper West Branch Brandywine Creek, HUC-12 (020402050202)**
    - `COMID = 4648450` is the first on the list

NOTE: sub-basin modeling results are accessed via the subtle __View subbasin attentuated results__ link, under the *Model > Water Quality* sub-tab.

```
NHD+ ComID | Area (ha)
4648450	263.61

Total Loads (not normalized):
  Sediment (kg/yr)
  Total Nitrogen (kg/yr)
  Total Phosphorus (kg/yr)  
	205,366.43	5,794.57	376.41

Loading Rates (area normalized):
  Sediment (kg/ha)
  Total Nitrogen (kg/ha)
  Total Phosphorus (kg/ha)  
	779.05	21.98	1.43

Loading Rates (area normalized):
  Sediment (mg/L)
  Total Nitrogen (mg/L)
  Total Phosphorus (mg/L)  
	162.76	4.59	0.3
```

In [16]:
# Example selecting by comid, which is the row index
base_reach_gdf.loc[4648450]

tp_conc                                                          0.2979
tn_conc                                                          4.5884
tss_conc                                                       162.5966
catchment_hectares                                             263.4373
watershed_hectares                                               263.61
maflowv                                                           1.412
geom                  (LINESTRING (-8449613.218716362 4882948.059213...
cluster                                        Brandywine and Christina
sub_focusarea                                                      <NA>
nord                                                              64639
nordstop                                                          64639
huc12                                                      020402050202
streamorder                                                           1
headwater                                                       

In [17]:
base_catch_gdf.loc[4648450]

tp_load                                                        375.8633
tn_load                                                       5789.5674
tss_load                                                    205160.7942
catchment_hectares                                             263.4373
watershed_hectares                                               263.61
tp_loadrate_ws                                                   1.4259
tn_loadrate_ws                                                  21.9625
tss_loadrate_ws                                                778.2738
maflowv                                                           1.412
geom_catchment        (POLYGON ((-8449229.677149855 4880762.40560297...
cluster                                        Brandywine and Christina
sub_focusarea                                                      <NA>
nord                                                              64639
nordstop                                                        

In [18]:
# Annual Areal Loading Rate, for Total Phosphorus (kg/ha)
base_catch_gdf.loc[4648450].tp_load / base_catch_gdf.loc[4648450].catchment_hectares 

1.4267656858007578

## Example Calcs

In [19]:
# Basic selection by column name
var = 'tp_load'
rest_catch_gdf[var]

comid
1748535        881.4618
1748537        296.6355
1748539        350.9217
1748541        517.2223
1748543        289.1315
                ...    
932040366     8244.2767
932040367     1771.2771
932040368        9.1347
932040369       65.8481
932040370    10648.8242
Name: tp_load, Length: 19496, dtype: float64

In [20]:
# Select Series by attribute
base_catch_gdf.tp_load

comid
1748535        881.4618
1748537        296.6355
1748539        350.9217
1748541        517.2223
1748543        289.1315
                ...    
932040366     8244.2767
932040367     1771.2771
932040368        9.1347
932040369       65.8481
932040370    10648.8242
Name: tp_load, Length: 19496, dtype: float64

In [21]:
# Calculate statistics for a data series
base_catch_gdf.tp_load.min()

0.0003

In [22]:
base_catch_gdf.tp_load.max()

266503.8003

In [23]:
# Calculate statistics for a data series, filtered by a categorical column.
base_reach_gdf[base_reach_gdf.cluster=='drb'].tp_conc.mean()

0.188712788679749

## Add Loading Rate Columns (kg/ha/y)

Only to Catchment dataframes `base_catch_gdf` and `rest_catch_gdf`.

In [24]:
# Define a function to loop through all the pollutants in a data frame
# for speed and ease that can also be applied to restoration dataframes.

def CalcLoadRate(df):
    for pollutant in ['tn','tp','tss']:
        df[f'{pollutant}_loadrate'] = df[f'{pollutant}_load'] / df.catchment_hectares
    return df

### to Base Model
... outputs for catchments.

In [25]:
# Apply to the base model run catchments
base_catch_gdf = CalcLoadRate(base_catch_gdf)

In [26]:
# Confirm values in Model My Watershed, from https://modelmywatershed.org/project/36183/
base_catch_gdf.loc[4648450].tp_loadrate

1.4267656858007578

In [27]:
base_catch_gdf.loc[4648450].tn_loadrate

21.977022236410715

In [28]:
base_catch_gdf.loc[4648450].tss_loadrate

778.7841516748008

In [29]:
base_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

### to Restoration Model 
... outputs for catchments.

In [30]:
# Apply to the restoration model run catchments
rest_catch_gdf = CalcLoadRate(rest_catch_gdf)

In [31]:
rest_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

### Confirm Differences

In [32]:
diff_df = base_catch_gdf.tp_load == rest_catch_gdf.tp_load
diff_df.value_counts()

True     19219
False      277
Name: tp_load, dtype: int64

In [33]:
diff_df = base_reach_gdf.tp_conc == rest_reach_gdf.tp_conc
diff_df.value_counts()

True     15943
False     3553
Name: tp_conc, dtype: int64

In [34]:
# Drop NaN values, which show up as False in comparisions
diff_df = base_reach_gdf.tp_conc.dropna() == rest_reach_gdf.tp_conc.dropna()
diff_df.value_counts()

True     15943
False      880
Name: tp_conc, dtype: int64

# Pollution Assessment Calculations

# Restoration Project Summary

In [ ]:
proj_rest_gdf.head()

,comid,practice_id,site_id,practice_type,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,practice_name,practice_description,project_name,project_status,creator_name,program_id,program_name,created_on,modified_on,practice_url,project_url,geom
0,2583213,5396,2864,Forest Buffer,2.8553,0.6454,331.0068,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((-8319500.216 5031043.008, -831..."
1,2583213,5399,2865,Forest Buffer,14.1300,3.4400,1438.7700,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-08-10,https://www.fielddoc.org/practices/5399,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((-8319473.054 5031829.104, -831..."
2,2583231,5396,2864,Forest Buffer,33.6847,7.6146,3904.9832,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((-8319500.216 5031043.008, -831..."
3,2583231,5435,2896,Forest Buffer,1.9100,0.4000,222.0400,Forest Buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Michelle DiBlasio,1,Delaware River Restoration Fund,2019-01-25,2021-08-10,https://www.fielddoc.org/practices/5435,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((-8318481.308 5031029.412, -831..."
4,2583231,52787,12863,Forest Buffer,16.4500,3.4600,1917.7000,sussex county fairgrounds buffer,buffer planting with NJ Youth Corps,EZG #44530 Synergistic Conservation Strategies...,complete,john parke,1,Delaware River Restoration Fund,2021-08-30,2021-08-30,https://www.fielddoc.org/practices/52787,https://www.fielddoc.org/projects/6368,"MULTIPOLYGON (((-8317478.876 5032359.115, -831..."


In [ ]:
proj_rest_gdf.project_status.value_counts()

complete    622
active      296
Name: project_status, dtype: int64

# Protection Project Summary

Protection projects exported from FieldDoc were subdived by NHDplus COMID.  
The Primary Key for this dataframe is thus:
- practice_i = FieldDoc (FD) primary key ID
- rn = Row number for number of polygons in the multipolygon submitted to FD as a project
- comid = NHDplus COMID intesecting with the protection polygon

**TO DO** in the future:
- Implement a Pandas/GeoPandas multi-index:  
https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

NOTE: For *Stage 1** the "avoided load" is taken as the Developed load - the Forested load. 
- Assumes 100% developement to Medium Intensity (NLCD class 23). 
- Only the average forested loading rate is provided in this table (averagess NLCD codes 41,42,43) in order to make the table simpler, however each land cover class' loading rate is explicitly used in the "avoided" load calculation.

In [ ]:
proj_prot_gdf.head()

,practice_i,practice_n,rn,comid,practice_t,practice_d,project_na,project_st,creator_na,program_id,...,parcel_devtssload_lbyr,parcel_foretssload_lbyr,parcel_tssload_lbyr_avoided,parcel_devtnload_lbyr,parcel_foretnload_lbyr,parcel_tnload_lbyr_avoided,parcel_devtpload_lbyr,parcel_foretpload_lbyr,parcel_tpload_lbyr_avoided,geom
0,5289,Bear Creek,1,4185445,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5,...,175779.645,9187.118,166592.527,205.600,17.768,187.832,76.994,4.230,72.764,"MULTIPOLYGON (((-8437443.527 5038594.992, -843..."
1,5289,Bear Creek,1,4185461,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5,...,159240.637,8322.707,150917.930,186.255,16.096,170.159,69.750,3.832,65.918,"MULTIPOLYGON (((-8436665.731 5039336.411, -843..."
2,5289,Bear Creek,1,4185483,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5,...,235688.406,12318.247,223370.159,275.672,23.823,251.849,103.235,5.672,97.563,"MULTIPOLYGON (((-8439309.894 5038692.974, -843..."
3,5289,Bear Creek,1,4185485,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5,...,48109.916,2514.463,45595.453,56.272,4.863,51.409,21.073,1.158,19.915,"MULTIPOLYGON (((-8438441.587 5037972.062, -843..."
4,5289,Bear Creek,1,4185505,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5,...,40026.507,2083.105,37943.402,34.930,3.776,31.154,16.804,0.944,15.860,"MULTIPOLYGON (((-8437800.078 5037986.266, -843..."


In [ ]:
proj_prot_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   practice_i                    354 non-null    Int64         
 1   practice_n                    354 non-null    category      
 2   rn                            354 non-null    int64         
 3   comid                         354 non-null    int64         
 4   practice_t                    354 non-null    category      
 5   practice_d                    74 non-null     category      
 6   project_na                    354 non-null    category      
 7   project_st                    354 non-null    category      
 8   creator_na                    354 non-null    category      
 9   program_id                    354 non-null    Int64         
 10  program_na                    354 non-null    category      
 11  created                 

## Sum by Project

*** THIS HAS ERRORS in the SUM!!!! ***

Anthony: I think I fixed this already, but it must not have saved!

In [ ]:
# Dissolve (Aggregate) by Protection Project
# Preselect colums to keep

proj_prot_projects_gdf = proj_prot_gdf.dissolve('project_na')

proj_prot_projects_gdf.area_acres.sum()

3180.486999999999

*** THIS HAS ERRORS in the SUM!!!! ***

In [ ]:
proj_prot_projects_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 52 entries, 16 Years LLC Project to Zemel Woodland South
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   geom                          52 non-null     geometry      
 1   practice_i                    52 non-null     Int64         
 2   practice_n                    52 non-null     category      
 3   rn                            52 non-null     int64         
 4   comid                         52 non-null     int64         
 5   practice_t                    52 non-null     category      
 6   practice_d                    15 non-null     category      
 7   project_st                    52 non-null     category      
 8   creator_na                    52 non-null     category      
 9   program_id                    52 non-null     Int64         
 10  program_na                    52 non-null     cat

In [ ]:
proj_prot_projects_gdf.area_acres.sum()

3180.486999999999

In [ ]:
# Save to CSV file
proj_prot_projects_gdf.iloc[:,1:].to_csv(data_folder /'proj_prot_projects.csv')

# Task 3a. Hotspots of Excess Pollution

**Stage 1 Task 3a. Identify “hotspots” across the basin where modeled NPS levels exceed the identified thresholds [targets] for healthy waters.**
```
    excess pollution = total pollution – threshold pollution
```

Tasks 3a & 3b form **DRWI Pollution Assement Goal 1**:
- Goal 1. Identify hotspots of excess nonpoint source pollution (nitrogen, phosphorus, sediment) in stream reaches and catchments of the Delaware River Watershed.


### Excess Load Rates

In [65]:
# # Explict use of Pandas/GeoPandas functionality

# # Create new columns
# base_catch_gdf['tn_loadrate_xs']  = base_catch_gdf.tn_loadrate  -  tn_loadrate_target
# base_catch_gdf['tp_loadrate_xs']  = base_catch_gdf.tp_loadrate  -  tp_loadrate_target
# base_catch_gdf['tss_loadrate_xs'] = base_catch_gdf.tss_loadrate -  tss_loadrate_target

In [90]:
# Define a function to loop through all the pollutants in a data frame
def CalcExcessLoadRate(df, targ_dict):
    for pollutant in ['tn', 'tp', 'tss']:
        df[f'{pollutant}_loadrate_xs'] = df[f'{pollutant}_loadrate'] - targ_dict[pollutant]['loadrate_target']
    return df

In [91]:
base_catch_gdf = CalcExcessLoadRate(base_catch_gdf, targets)

In [92]:
base_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

In [93]:
# Confirm calculation relative to Model My Watershed
base_catch_gdf.loc[4648450].tp_loadrate

1.4267656858007578

In [94]:
base_catch_gdf.loc[4648450].tp_loadrate_xs

1.1167656858007577

In [95]:
# Difference should equal target value
targets['tp']['loadrate_target']

0.31

Check for negative values, which will create an error with log-normalized data.

In [72]:
base_catch_gdf.tp_loadrate_xs.min()

-0.30755539616587657

In [73]:
base_catch_gdf.tn_loadrate_xs.min()

-17.067776542523625

In [74]:
base_catch_gdf.tss_loadrate_xs.min()

-914.8911871003613

In [75]:
test = base_catch_gdf.tss_loadrate_xs.copy()
test.min()

-914.8911871003613

In [76]:
tss_loadrate_target / 100

9.238

In [77]:
test[test < tss_loadrate_target / 100] = tss_loadrate_target / 100
test.min()

9.238

In [78]:
test.size

19496

In [79]:
test.value_counts()

9.238000       16962
1196.651560        1
486.614431         1
956.691306         1
696.480269         1
               ...  
508.270032         1
1373.249977        1
1274.941564        1
182.060080         1
385.259959         1
Name: tss_loadrate_xs, Length: 2535, dtype: int64

### Excess Concentrations

In [80]:
# # Explict use of Pandas/GeoPandas functionality

# # Create new columns
# base_gdf_proj['tn_conc_xs']  = base_gdf_proj.tn_conc  -  tn_conc_target
# base_gdf_proj['tp_conc_xs']  = base_gdf_proj.tp_conc  -  tp_conc_target
# base_gdf_proj['tss_conc_xs'] = base_gdf_proj.tss_conc -  tss_conc_target

In [87]:
# Define a function to loop through all the pollutants in a data frame
def CalcExcessConcs(df, targ_dict):
    for pollutant in ['tn', 'tp', 'tss']:
        df[f'{pollutant}_conc_xs'] = df[f'{pollutant}_conc'] - targ_dict[pollutant]['conc_target']
    return df

In [88]:
base_reach_gdf = CalcExcessConcs(base_reach_gdf, targets)

In [89]:
base_reach_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             16823 non-null  float64 
 1   tn_conc             16823 non-null  float64 
 2   tss_conc            16823 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19494 non-null  geometry
 7   cluster             17358 non-null  category
 8   sub_focusarea       186 non-null    Int64   
 9   nord                18870 non-null  Int64   
 10  nordstop            18844 non-null  Int64   
 11  huc12               19496 non-null  category
 12  streamorder         19496 non-null  int64   
 13  headwater           19496 non-null  int64   
 14  phase               4082 non-null   category
 15  fa_name           

In [125]:
base_reach_gdf.loc[4648450].tp_conc

0.2979

In [126]:
base_reach_gdf.loc[4648450].tp_conc_xs

0.2079

In [127]:
targets['tp']['conc_target']

0.09

Check for negative values, which will create an error with log-normalized data.

In [128]:
base_reach_gdf.tp_conc_xs.min()

-0.089

In [129]:
base_reach_gdf.tn_conc_xs.min()

-4.7248

In [130]:
base_reach_gdf.tss_conc_xs.min()

-236.4147

# Task 3b. Hotspots of Excess Non Point Source Pollution

**Stage 1 Task 3b. Estimate the amount of excess nonpoint and point source pollution contributing to each hotspot** 
- ...and identify subwatersheds where point source loads override the ability of nonpoint source pollution interventions (i.e., non-urban BMPs) to achieve the “established” DRWI healthy water thresholds.

```
    excess nonpoint source pollution = excess pollution – point source pollution
```

Tasks 3a & 3b form **DRWI Pollution Assement Goal 1**:
- Goal 1. Identify hotspots of excess nonpoint source pollution (nitrogen, phosphorus, sediment) in stream reaches and catchments of the Delaware River Watershed.

### Point Source Loads by NPDES_ID



In [99]:
point_src_gdf.head()

,ogc_fid,geom,city,state,latitude,longitude,huc12,avg_n_conc,lbsn_yr,mgd,avgpconc,lbsp_yr,kgn_yr,kgp_yr,facilityname,comid
npdes_id,,,,,,,,,,,,,,,,
PA0033995,1,POINT (-8461641.847 4920586.837),BERN TWP,PA,40.375000,-76.012222,020402030409,0.191,84.591269,0.163050,0.191,1325.042759,38.369923,601.028795,COUNTY OF BERKS WWTP,4783187
PA0051811,1,POINT (-8414640.309 4954415.583),SOUTH WHITEHALL1TWP,PA,40.606111,-75.590000,020401060703,35.000,32.011082,0.000300,35.000,2.733729,14.519971,1.239998,LEHIGH COUNTY AUTH,4187751
1594403,1,POINT (-8423298.405 4881252.661),WEST VINCENT TWP,PA,40.105277,-75.667777,020402031003,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,MATTHEWS MEADOWS STP,4781791
1592417,1,POINT (-8421319.479 4881171.885),WEST VINCENT TWP,PA,40.104722,-75.650000,020402031003,0.000,0.000000,0.000000,0.000,NaN,0.000000,NaN,SAINT STEPHEN'S GREENE STP,4782621
NJ0065196,1,POINT (-8349522.329 4975910.787),Township of Washington,NJ,40.752548,-75.005035,020401050401,7.175,155.557987,0.000677,7.175,5.599735,70.559859,2.539995,390 RT 57,2588253


In [100]:
point_src_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 812 entries, PA0033995 to NJ0104388
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ogc_fid       812 non-null    int64   
 1   geom          812 non-null    geometry
 2   city          789 non-null    category
 3   state         812 non-null    category
 4   latitude      812 non-null    float64 
 5   longitude     812 non-null    float64 
 6   huc12         812 non-null    category
 7   avg_n_conc    811 non-null    float64 
 8   lbsn_yr       811 non-null    float64 
 9   mgd           812 non-null    float64 
 10  avgpconc      811 non-null    float64 
 11  lbsp_yr       809 non-null    float64 
 12  kgn_yr        811 non-null    float64 
 13  kgp_yr        809 non-null    float64 
 14  facilityname  812 non-null    category
 15  comid         812 non-null    int64   
dtypes: category(4), float64(9), geometry(1), int64(2)
memory usage: 165.5 

In [101]:
# Confirm values relative to Model My Watershed, using Analyze tab.
# Use Upper West Branch Brandywine Creek, HUC-12 (020402050202). https://modelmywatershed.org/project/36183/.

point_src_gdf.loc['PA0026859']

ogc_fid                                                    1
geom            POINT (-8440645.766627265 4861987.095563347)
city                                   SOUTH COATESVILLEBORO
state                                                     PA
latitude                                           39.972777
longitude                                         -75.823611
huc12                                           020402050202
avg_n_conc                                             3.928
lbsn_yr                                         42772.604237
mgd                                                     4.58
avgpconc                                               3.928
lbsp_yr                                          3290.196934
kgn_yr                                          19401.311101
kgp_yr                                           1492.407008
facilityname                                 COATESVILLE STP
comid                                              932040160
Name: PA0026859, dtype: 

**These `kgn_yr` & `kgp_yr` values match those in the MMW Analyze > Pt Sources tab!**  
They are not attenuated.

In [102]:
# Look at all point sources in this COMID
point_src_gdf[point_src_gdf['comid']==932040160]

,ogc_fid,geom,city,state,latitude,longitude,huc12,avg_n_conc,lbsn_yr,mgd,avgpconc,lbsp_yr,kgn_yr,kgp_yr,facilityname,comid
npdes_id,,,,,,,,,,,,,,,,
PA0026859,1,POINT (-8440645.767 4861987.096),SOUTH COATESVILLEBORO,PA,39.972777,-75.823611,020402050202,3.928,42772.604237,4.5800,3.928,3290.196934,19401.311101,1492.407008,COATESVILLE STP,932040160
PA0036897,1,POINT (-8438836.825 4861026.832),MODENABORO,PA,39.966166,-75.807361,020402050202,4.180,2153.098031,0.1615,4.180,698.864540,976.628042,316.999364,SOUTH COATESVILLE STP,932040160


### Point Source Loads by COMID

In [103]:
# Many COMID's have more than one NPDES-permitted point source
point_src_gdf.comid.value_counts()

4782585    7
4782471    4
2586879    4
4496204    4
4481865    4
          ..
4481751    1
2583323    1
9482442    1
2617298    1
2587723    1
Name: comid, Length: 672, dtype: int64

In [104]:
# Sum loads by COMID groups
# Non-summable dtypes (object, category, geometry) will be dropped automatically
temp_df = point_src_gdf.groupby('comid').sum()

# Other fields that should not be summed, such as lat/lon or concentrations, need to dropped explicitly
point_src_loads_comid_df = temp_df.drop(['ogc_fid',
                                         'latitude',
                                         'longitude',
                                         'avg_n_conc',
                                         'avgpconc',
                                        ], axis=1) 
point_src_loads_comid_df

,lbsn_yr,mgd,lbsp_yr,kgn_yr,kgp_yr
comid,,,,,
1750459,10976.163640,0.277000,2533.020195,4978.700018,1148.957696
2583201,831.406294,0.007388,29.894647,377.119244,13.559973
2583217,6.437490,0.011000,86.862028,2.919994,39.399921
2583283,2880.512400,0.032000,411.492323,1306.577380,186.649626
2583321,452.233701,0.004239,16.270096,205.129589,7.379985
...,...,...,...,...,...
932040308,25928.094896,3.145000,28987.931086,11760.776420,13148.693637
932040358,34139.312317,0.380000,2926.236218,15485.318952,1327.317339
932040366,328489.967326,3.784100,28332.960531,149000.421260,12851.604233


In [105]:
# Confirm values relative to Model My Watershed, using Model tab.
# Use Upper West Branch Brandywine Creek, HUC-12 (020402050202). https://modelmywatershed.org/project/36183/.

# **These values are bigger than the MMW Model totals for COMID!**
point_src_loads_comid_df.loc[932040160]

lbsn_yr    44925.702268
mgd            4.741500
lbsp_yr     3989.061474
kgn_yr     20377.939143
kgp_yr      1809.406372
Name: 932040160, dtype: float64

In [106]:
## These values match the MMW totals for COMID!
base_catch_gdf.tn_load.loc[932040160]

18661.1301

In [107]:
## These values match the MMW totals for COMID!
base_catch_gdf.tp_load.loc[932040160]

1281.7992

See MMW documentation: 
- https://wikiwatershed.org/help/model-help/mmw-tech/#description-and-editing-of-key-model-input-data-and-parameters, and scroll to:
  - Settings > Other Model Data
    - Figure 8
    - Wetland / Water Filtration Factor
    - N Wetland / Water Retention Fraction
    - P Wetland / Water Retention Fraction
    - TSS Wetland / Water Retention Fraction

### Point Source Loads by HUC12

Compare to **Upper West Branch Brandywine Creek, HUC-12 (020402050202)**, https://modelmywatershed.org/project/36219/

```
Sources					Sediment		Total Nitrogen	Total Phosphorus	
Total Loads (kg)		10,461,802.1		286,239.1		15,225.7
Loading Rates (kg/ha)		822.10			22.49			1.20

Point Sources (kg)			0.0			20,689.0		1,377.0
```

In [108]:
# Sum loads by COMID groups
# Non-summable dtypes (object, category, geometry) will be dropped automatically
temp_df = point_src_gdf.groupby('huc12').sum()

# Other fields that should not be summed, such as lat/lon or concentrations, need to dropped explicitly
point_src_loads_huc12_df = temp_df.drop(['ogc_fid',
                                         'latitude',
                                         'longitude',
                                         'avg_n_conc',
                                         'avgpconc',
                                         'comid'
                                        ], axis=1) 
point_src_loads_huc12_df

,lbsn_yr,mgd,lbsp_yr,kgn_yr,kgp_yr
huc12,,,,,
020401010201,66998.423846,0.628000,38.294249,30389.949069,17.369965
020401010204,15.145739,0.000088,1.300726,6.869986,0.589999
020401010205,29558.795389,1.010000,785.991122,13407.633118,356.519285
020401010206,616.962907,0.006643,52.866788,279.849439,23.979952
020401010304,486.228941,0.291000,2926.236218,220.549558,1327.317339
...,...,...,...,...,...
020402060703,233.513350,0.585500,342.223163,105.919788,155.229689
020402070101,5654.321191,0.091000,484.663661,2564.754858,219.839559
020402070405,182227.231248,11.400000,14769.344627,82656.814276,6699.256568


In [109]:
point_src_loads_huc12_df.size

1305

#### Compare with MMW sub-basin modeling 
- **West Branch Brandywine Creek, HUC-10**, 
  - available at https://modelmywatershed.org/project/36183/.
  - Upper West Branch Brandywine Creek, HUC-12 (020402050202)
    - `COMID = 4648450` is the first on the list

In [110]:
# Non-Attenuated Point Source loads
point_src_loads_huc12_df.kgn_yr['020402050202']

28514.552829347533

In [111]:
# Attenuated Point Source loads, which matches MMW when running HUC12 alone (different than in subbasin mode for P)
mmw_huc12_loads_df.tn_pt_source_kg['020402050202']

20689.0

In [112]:
# Nitrogen attenuation fraction, not retained (i.e. attenuated load / raw load)
mmw_huc12_loads_df.tn_pt_source_kg['020402050202'] / point_src_loads_huc12_df.kgn_yr['020402050202']

0.7255593353968582

In [113]:
# Phosphorus attenuation, fraction not retained
mmw_huc12_loads_df.tp_pt_source_kg['020402050202'] / point_src_loads_huc12_df.kgp_yr['020402050202']

0.30112045468342075

### TO DO: Add Fraction Attenuated to COMID calcualted from HUC12

**NOT COMPLETED**: The attenuation fractions I calculated are much too variable to make sense, and some are greater than 1!

In [114]:
# Calcuate attenuation fraction
point_src_loads_huc12_df['tn_load_ps_atn_frac'] = mmw_huc12_loads_df.tn_pt_source_kg / point_src_loads_huc12_df.kgn_yr
point_src_loads_huc12_df['tp_load_ps_atn_frac'] = mmw_huc12_loads_df.tp_pt_source_kg / point_src_loads_huc12_df.kgp_yr

In [115]:
point_src_loads_huc12_df

,lbsn_yr,mgd,lbsp_yr,kgn_yr,kgp_yr,tn_load_ps_atn_frac,tp_load_ps_atn_frac
huc12,,,,,,,
020401010201,66998.423846,0.628000,38.294249,30389.949069,17.369965,0.042086,2.590679
020401010204,15.145739,0.000088,1.300726,6.869986,0.589999,1.004369,1.016951
020401010205,29558.795389,1.010000,785.991122,13407.633118,356.519285,0.501431,0.123415
020401010206,616.962907,0.006643,52.866788,279.849439,23.979952,0.003573,0.041702
020401010304,486.228941,0.291000,2926.236218,220.549558,1327.317339,0.999775,0.999987
...,...,...,...,...,...,...,...
020402060703,233.513350,0.585500,342.223163,105.919788,155.229689,0.000000,0.000000
020402070101,5654.321191,0.091000,484.663661,2564.754858,219.839559,0.071742,0.081878
020402070405,182227.231248,11.400000,14769.344627,82656.814276,6699.256568,1.000000,1.000006


### Merge Point Source Load & LoadRate Columns to Modeled Catchment Outputs

Using raw point source loads, not attenuated, due to issues noted in block above.

In [116]:
# # Explict use of Pandas/GeoPandas functionality
# This simple merge/join is possible, because both dataframes share the same index (comid)

base_catch_gdf['tn_load_ps'] = point_src_loads_comid_df.kgn_yr
base_catch_gdf['tp_load_ps'] = point_src_loads_comid_df.kgp_yr 

In [117]:
# Fill NaN with 0, because arithmetric with a NaN produces NaN!
base_catch_gdf[['tn_load_ps','tp_load_ps']] = base_catch_gdf[['tn_load_ps','tp_load_ps']].fillna(0)

In [118]:
base_catch_gdf[['tp_load','tp_load_ps']].loc[932040160]

tp_load       1281.799200
tp_load_ps    1809.406372
Name: 932040160, dtype: float64

In [119]:
base_catch_gdf[['tp_load','tp_load_ps']].loc[4648450]

tp_load       375.8633
tp_load_ps      0.0000
Name: 4648450, dtype: float64

In [120]:
base_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

### Excess NPS Load Rates

**NOTE**: These are underestimates of Non Point Source Loading Rates, because the point source loads are not attenuated.

In [121]:
# # Explict use of Pandas/GeoPandas functionality

base_catch_gdf['tn_loadrate_xsnps']  = ((base_catch_gdf.tn_load - base_catch_gdf.tn_load_ps
                                        )  / base_catch_gdf.catchment_hectares
                                       ) - tn_loadrate_target

base_catch_gdf['tp_loadrate_xsnps']  = ((base_catch_gdf.tp_load - base_catch_gdf.tp_load_ps
                                        )  / base_catch_gdf.catchment_hectares
                                       ) - tp_loadrate_target
# TSS doesn't come from NPDES point sources

In [122]:
base_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

Confirm values make sense for COMIDs with:
- no point sources: `4648450`
- large point sources: `932040160`

In [123]:
base_catch_gdf[['tn_loadrate',
                'tn_loadrate_xs',
                'tn_loadrate_xsnps',
               ]].loc[4648450]

tn_loadrate          21.977022
tn_loadrate_xs        4.907022
tn_loadrate_xsnps     4.907022
Name: 4648450, dtype: float64

In [124]:
base_catch_gdf[['tp_loadrate',
                'tp_loadrate_xs',
                'tp_loadrate_xsnps',
               ]].loc[4648450]

tp_loadrate          1.426766
tp_loadrate_xs       1.116766
tp_loadrate_xsnps    1.116766
Name: 4648450, dtype: float64

In [125]:
base_catch_gdf[['tn_loadrate',
                'tn_loadrate_xs',
                'tn_loadrate_xsnps',
               ]].loc[932040160]

tn_loadrate          37.486353
tn_loadrate_xs       20.416353
tn_loadrate_xsnps   -20.518714
Name: 932040160, dtype: float64

In [126]:
base_catch_gdf[['tp_loadrate',
                'tp_loadrate_xs',
                'tp_loadrate_xsnps',
               ]].loc[932040160]

tp_loadrate          2.574870
tp_loadrate_xs       2.264870
tp_loadrate_xsnps   -1.369854
Name: 932040160, dtype: float64

# Task 3c1. Remaining Pollution after Restoration

**Stage 1 Task 3.c. What proportion of the NPS within the cluster boundaries could be addressed by DRWI land protection and ag restoration tactics?”**
- In order to address the question *“What proportion of the NPS within the cluster boundaries could be addressed by DRWI land protection and ag restoration tactics?”*,
- The remaining load (non-point source load) that is in excess of the “healthy WQ” criteria will be compared to the cumulative load reductions from all DRWI BMP investments that have been tracked to date (reported to FieldDocs).

Tasks 3c & 3d form **DRWI Pollution Assement Goal 2a**:
- Goal 2. Quantify progress toward improving water quality by DRWI-style land protection and restoration activities.
  - a. What fraction of excess nonpoint source pollution has already been reduced?
  - b. What level of investment is still required to achieve acceptable water quality?


## Reduction by Restoration

Calculate the Restoration Reduction, as the difference between the base model run and the restoration run.

In [132]:
# Concentration reductions for Reaches
base_reach_gdf['tn_conc_red']  = base_reach_gdf.tn_conc  - rest_reach_gdf.tn_conc
base_reach_gdf['tp_conc_red']  = base_reach_gdf.tp_conc  - rest_reach_gdf.tp_conc
base_reach_gdf['tss_conc_red'] = base_reach_gdf.tss_conc - rest_reach_gdf.tss_conc

In [133]:
# Load Rate reductions for Catchments
base_catch_gdf['tn_loadrate_red']  = base_catch_gdf.tn_loadrate  - rest_catch_gdf.tn_loadrate
base_catch_gdf['tp_loadrate_red']  = base_catch_gdf.tp_loadrate  - rest_catch_gdf.tp_loadrate
base_catch_gdf['tss_loadrate_red'] = base_catch_gdf.tss_loadrate - rest_catch_gdf.tss_loadrate

In [134]:
base_catch_gdf.iloc[:,-3:].max()

tn_loadrate_red       192.491960
tp_loadrate_red        40.494407
tss_loadrate_red    26531.917615
dtype: float64

## Avoided by Protection

Based on protection project data uploaded into FieldDoc. Protection projects were exploaded from multi-polygon to polygon then intersected with NHDplus COMID.

The avoided load is taken as the Developed load - the Forested load. Assumes 100% developement to Medium Intensity (NLCD class 23). Only the average forested loading rate is provided in this table (averagess NLCD codes 41,42,43) in order to make the table simpler, however each land cover class' loading rate is explicitly used in the "avoided" load calculation.

Primary Key
- practice_i = FieldDoc (FD) primary key ID
- rn = Row number for number of polygons in the multipolygon submitted to FD as a project
- comid = NHDplus COMID intesecting with the protection polygon

SQL:
```sql
CONSTRAINT pk_protection_lbsavoided_fd PRIMARY KEY (practice_i, rn, comid)
```

**TO DO** in the future:
- Implement a Pandas/GeoPandas multi-index:  
https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

In [135]:
proj_prot_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   practice_i                    354 non-null    Int64         
 1   practice_n                    354 non-null    category      
 2   rn                            354 non-null    int64         
 3   comid                         354 non-null    int64         
 4   practice_t                    354 non-null    category      
 5   practice_d                    74 non-null     category      
 6   project_na                    354 non-null    category      
 7   project_st                    354 non-null    category      
 8   creator_na                    354 non-null    category      
 9   program_id                    354 non-null    Int64         
 10  program_na                    354 non-null    category      
 11  created                 

In [136]:
# Sum loads by COMID
# Preselect colums to keep
proj_prot_loads_comid_df = proj_prot_gdf.iloc[:, [3, 16, 31, 28, 25, ]
                             ].groupby('comid').sum()

proj_prot_loads_comid_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 228 entries, 2583463 to 932040274
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   area_acres                   228 non-null    float64
 1   parcel_tpload_lbyr_avoided   228 non-null    float64
 2   parcel_tnload_lbyr_avoided   228 non-null    float64
 3   parcel_tssload_lbyr_avoided  228 non-null    float64
dtypes: float64(4)
memory usage: 8.9 KB


In [137]:
# Pollution Avoided
# Convert to kg/ha, where `1 kg/ha = 0.89218 lb/ac`, http://www.kylesconverter.com/area-density/kilograms-per-hectare-to-pounds-per-acre

base_catch_gdf['tn_load_avoid']  = proj_prot_loads_comid_df.parcel_tnload_lbyr_avoided  / 0.89218
base_catch_gdf['tp_load_avoid']  = proj_prot_loads_comid_df.parcel_tpload_lbyr_avoided  / 0.89218
base_catch_gdf['tss_load_avoid'] = proj_prot_loads_comid_df.parcel_tssload_lbyr_avoided / 0.89218

In [138]:
# Fill NaN with 0, because arithmetric with a NaN produces NaN!
base_catch_gdf[['tn_load_avoid','tp_load_avoid','tss_load_avoid']] = base_catch_gdf[['tn_load_avoid','tp_load_avoid','tss_load_avoid']].fillna(0)

In [139]:
base_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

## Remaining Pollution

In [140]:
# Remaining Excess Concentration for Reaches
# NOTE: In Stage 1, we can not subtract non point source concentrations
base_reach_gdf['tn_conc_rem']  = base_reach_gdf.tn_conc_xs  - base_reach_gdf.tn_conc_red
base_reach_gdf['tp_conc_rem']  = base_reach_gdf.tp_conc_xs  - base_reach_gdf.tp_conc_red
base_reach_gdf['tss_conc_rem'] = base_reach_gdf.tss_conc_xs - base_reach_gdf.tss_conc_red

In [141]:
# Remaining Excess NPS Load Rates for Reaches
base_catch_gdf['tn_loadrate_rem']  = base_catch_gdf.tn_loadrate_xsnps - base_catch_gdf.tn_loadrate_red
base_catch_gdf['tp_loadrate_rem']  = base_catch_gdf.tp_loadrate_xsnps - base_catch_gdf.tp_loadrate_red
base_catch_gdf['tss_loadrate_rem'] = base_catch_gdf.tss_loadrate_xs   - base_catch_gdf.tss_loadrate_red

# Task 3c2. Proportion Remaining after Restoration

**Stage 1 Task 3.c. What proportion of the NPS within the cluster boundaries could be addressed by DRWI land protection and ag restoration tactics?”**
- In order to address the question *“What proportion of the NPS within the cluster boundaries could be addressed by DRWI land protection and ag restoration tactics?”*,
- The remaining load (non-point source load) that is in excess of the “healthy WQ” criteria will be compared to the cumulative load reductions from all DRWI BMP investments that have been tracked to date (reported to FieldDocs).

Tasks 3c & 3d form **DRWI Pollution Assement Goal 2a**:
- Goal 2. Quantify progress toward improving water quality by DRWI-style land protection and restoration activities.
  - a. What fraction of excess nonpoint source pollution has already been reduced?
  - b. What level of investment is still required to achieve acceptable water quality?

## Proportion Remaining

Add columns and set to zero any with  excess NPS pollution < 0.

In [161]:
# Proportion Remaining of Excess NPS Concentration for Reaches
# NOTE: In Stage 1, we can not subtract non point source concentrations
base_reach_gdf['tn_conc_rem_p']  = base_reach_gdf.tn_conc_rem / base_reach_gdf.tn_conc_xs
base_reach_gdf['tp_conc_rem_p']  = base_reach_gdf.tp_conc_rem / base_reach_gdf.tp_conc_xs
base_reach_gdf['tss_conc_rem_p'] = base_reach_gdf.tss_conc_rem / base_reach_gdf.tss_conc_xs

In [162]:
# Proportion Remaining of Excess NPS Load Rates for Catchments
base_catch_gdf['tn_loadrate_rem_p']  = base_catch_gdf.tn_loadrate_rem / base_catch_gdf.tn_loadrate_xsnps
base_catch_gdf['tp_loadrate_rem_p']  = base_catch_gdf.tp_loadrate_rem / base_catch_gdf.tp_loadrate_xsnps
base_catch_gdf['tss_loadrate_rem_p'] = base_catch_gdf.tss_loadrate_rem / base_catch_gdf.tss_loadrate_xs  # TSS has no point sources

In [163]:
mask_reach = base_reach_gdf['tp_conc_xs'] < 0
mask_reach.value_counts()

True     11481
False     8015
Name: tp_conc_xs, dtype: int64

In [164]:
# Set proportion = 0 where Excess < 0
for pollutant in ['tn', 'tp', 'tss']:
    mask_reach = base_reach_gdf[f'{pollutant}_conc_xs'] < 0
    base_reach_gdf.loc[mask_reach, f'{pollutant}_conc_rem_p'] = 0
    
    if pollutant != 'tss':
        mask_catch = base_catch_gdf[f'{pollutant}_loadrate_xsnps'] < 0
    else:
        mask_catch = base_catch_gdf[f'{pollutant}_loadrate_xs'] < 0
    base_catch_gdf.loc[mask_catch, f'{pollutant}_loadrate_rem_p'] = 0
       

Confirm outputs

In [165]:
base_reach_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             16823 non-null  float64 
 1   tn_conc             16823 non-null  float64 
 2   tss_conc            16823 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19494 non-null  geometry
 7   cluster             17358 non-null  category
 8   sub_focusarea       186 non-null    Int64   
 9   nord                18870 non-null  Int64   
 10  nordstop            18844 non-null  Int64   
 11  huc12               19496 non-null  category
 12  streamorder         19496 non-null  int64   
 13  headwater           19496 non-null  int64   
 14  phase               4082 non-null   category
 15  fa_name           

In [166]:
base_reach_gdf.loc[:,['cluster','phase','fa_name']].dropna()

,cluster,phase,fa_name
comid,,,
2583185,New Jersey Highlands,Phase 1,Upper Paulins Kill
2583187,New Jersey Highlands,Phase 2,Paulinskill
2583189,New Jersey Highlands,Phase 1,Upper Paulins Kill
2583191,New Jersey Highlands,Phase 1,Upper Paulins Kill
2583193,New Jersey Highlands,Phase 2,Paulinskill
...,...,...,...
932040305,Upper Lehigh,Phase 1,Bear Creek
932040306,Upper Lehigh,Phase 2,Bear Creek
932040311,Kirkwood - Cohansey Aquifer,Phase 1,Core Pine Barrens


In [167]:
base_reach_gdf[base_reach_gdf.fa_name=='Paulinskill']

,tp_conc,tn_conc,tss_conc,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,...,tss_conc_xs,tn_conc_red,tp_conc_red,tss_conc_red,tn_conc_rem,tp_conc_rem,tss_conc_rem,tn_conc_rem_p,tp_conc_rem_p,tss_conc_rem_p
comid,,,,,,,,,,,,,,,,,,,,,
2583187,0.0442,0.3627,119.2618,4.1367,4.14,0.026,"MULTILINESTRING ((-8322548.781 5037669.293, -8...",New Jersey Highlands,<NA>,70080,...,-118.0382,0.0,0.0,0.0000,-4.3673,-0.0458,-118.0382,0.0,0.0,0.0
2583193,0.0302,0.1657,76.9305,72.0329,72.09,0.460,"MULTILINESTRING ((-8325769.573 5035889.865, -8...",New Jersey Highlands,<NA>,70077,...,-160.3695,0.0,0.0,0.0000,-4.5643,-0.0598,-160.3695,0.0,0.0,0.0
2583195,0.0838,0.5955,87.1572,244.9663,245.16,1.537,"MULTILINESTRING ((-8322125.010 5036964.971, -8...",New Jersey Highlands,<NA>,70082,...,-150.1428,0.0,0.0,0.0000,-4.1345,-0.0062,-150.1428,0.0,0.0,0.0
2583197,0.0270,0.1993,57.7699,40.3780,1427.94,10.051,"MULTILINESTRING ((-8323708.819 5034362.050, -8...",New Jersey Highlands,<NA>,70073,...,-179.5301,0.0,0.0,0.0009,-4.5307,-0.0630,-179.5310,0.0,0.0,0.0
2583199,0.0274,0.1928,65.7503,215.1096,1018.44,7.164,"MULTILINESTRING ((-8324613.787 5035722.935, -8...",New Jersey Highlands,<NA>,70074,...,-171.5497,0.0,0.0,0.0012,-4.5372,-0.0626,-171.5509,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24944708,0.0134,0.1045,31.9760,10.7015,508.59,3.667,"MULTILINESTRING ((-8328851.742 5033000.233, -8...",New Jersey Highlands,<NA>,70034,...,-205.3240,0.0,0.0,0.0000,-4.6255,-0.0766,-205.3240,0.0,0.0,0.0
24944722,0.1364,1.8364,91.8525,55.3961,55.44,0.363,"MULTILINESTRING ((-8324378.673 5020540.248, -8...",New Jersey Highlands,<NA>,70109,...,-145.4475,0.0,0.0,0.0000,-2.8936,0.0464,-145.4475,0.0,1.0,0.0
24944724,0.1189,1.6361,82.8757,120.1448,229.95,1.500,"MULTILINESTRING ((-8323633.218 5021222.916, -8...",New Jersey Highlands,<NA>,70107,...,-154.4243,0.0,0.0,0.0000,-3.0939,0.0289,-154.4243,0.0,1.0,0.0


In [168]:
base_catch_gdf[base_catch_gdf.fa_name=='Paulinskill']

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,...,tss_loadrate_red,tn_load_avoid,tp_load_avoid,tss_load_avoid,tn_loadrate_rem,tp_loadrate_rem,tss_loadrate_rem,tn_loadrate_rem_p,tp_loadrate_rem_p,tss_loadrate_rem_p
comid,,,,,,,,,,,,,,,,,,,,,
2583187,1.0270,8.4276,2770.9126,4.1367,4.14,0.2481,2.0355,669.3026,0.026,"MULTIPOLYGON (((-8322349.220 5037276.816, -832...",...,0.0,0.0,0.0,0.0,-15.032724,-0.061734,-253.963512,0.0,0.0,0.0
2583193,12.4116,68.1305,31623.0859,72.0329,72.09,0.1722,0.9448,438.6615,0.460,"MULTIPOLYGON (((-8325931.539 5035114.436, -832...",...,0.0,0.0,0.0,0.0,-16.124175,-0.137695,-484.791076,0.0,0.0,0.0
2583195,115.0439,817.8438,119708.6198,244.9663,245.16,0.4695,3.3362,488.2876,1.537,"MULTIPOLYGON (((-8322884.353 5034593.476, -832...",...,0.0,0.0,0.0,0.0,-13.731403,0.159632,-435.126171,0.0,1.0,0.0
2583197,9.5954,104.1021,13754.6282,40.3780,1427.94,0.1698,1.2536,363.3700,10.051,"MULTIPOLYGON (((-8323006.964 5033524.219, -832...",...,0.0,0.0,0.0,0.0,-14.491811,-0.072361,-583.153405,0.0,0.0,0.0
2583199,55.1941,375.0488,111785.3128,215.1096,1018.44,0.1722,1.2119,413.3006,7.164,"MULTIPOLYGON (((-8325931.539 5035114.436, -832...",...,0.0,0.0,0.0,0.0,-15.326476,-0.053414,-404.133222,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24944708,0.8440,5.5244,2155.7534,10.7015,508.59,0.0863,0.6733,206.0228,3.667,"MULTIPOLYGON (((-8328860.039 5032927.198, -832...",...,0.0,0.0,0.0,0.0,-16.553773,-0.231133,-722.355959,0.0,0.0,0.0
24944722,44.2554,595.6882,29795.1576,55.3961,55.44,0.7981,10.7448,537.4308,0.363,"MULTIPOLYGON (((-8324408.639 5019389.693, -832...",...,0.0,0.0,0.0,0.0,-6.316748,0.488890,-385.943407,0.0,1.0,0.0
24944724,75.6097,1062.9409,58215.4341,120.1448,229.95,0.6931,9.5371,483.0957,1.500,"MULTIPOLYGON (((-8325298.285 5020521.972, -832...",...,0.0,0.0,0.0,0.0,-8.222835,0.319321,-439.256066,0.0,1.0,0.0


In [169]:
base_catch_gdf[base_catch_gdf.cluster=='Paulinskill'][base_catch_gdf.tp_loadrate_red>0].iloc[:,19:]

C:\Users\sjordan\Anaconda3\envs\WikiSRAT\lib\site-packages\geopandas\geodataframe.py:1299: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


,tn_loadrate,tp_loadrate,tss_loadrate,tn_loadrate_xs,tp_loadrate_xs,tss_loadrate_xs,tn_load_ps,tp_load_ps,tn_loadrate_xsnps,tp_loadrate_xsnps,...,tss_loadrate_red,tn_load_avoid,tp_load_avoid,tss_load_avoid,tn_loadrate_rem,tp_loadrate_rem,tss_loadrate_rem,tn_loadrate_rem_p,tp_loadrate_rem_p,tss_loadrate_rem_p
comid,,,,,,,,,,,,,,,,,,,,,


In [170]:
focusarea_gdf[focusarea_gdf.cluster=='New Jersey Highlands']

,cluster,strategy,nord,phase,geom,change,tier,subfocusare,area_acres,name_final,geom_buff
name,,,,,,,,,,,
Beaver Brook,New Jersey Highlands,preservation,6209,Phase 2,"MULTIPOLYGON (((-8330061.372 5007250.293, -833...",None,None,None,11108.7399,Beaver Brook,01060000206A7F000001000000010300000001000000B1...
Lopatcong Creek,New Jersey Highlands,hybrid,6060,Phase 2,"MULTIPOLYGON (((-8359813.292 4975422.920, -835...",None,None,None,9581.9444,Lopatcong Creek,01060000206A7F00000100000001030000000100000036...
Lower Middle Musconetcong,New Jersey Highlands,hybrid,5859,Phase 2,"MULTIPOLYGON (((-8328023.381 4990277.593, -832...",None,None,None,35954.1793,Lower Middle Musconetcong,01060000206A7F00000100000001030000000100000042...
Paulinskill,New Jersey Highlands,hybrid,6471,Phase 2,"MULTIPOLYGON (((-8338217.043 5013234.547, -833...",None,None,None,68203.4122,Paulinskill,01060000206A7F000001000000010300000004000000B1...
Upper Musconetcong,New Jersey Highlands,preservation,5895,Phase 2,"MULTIPOLYGON (((-8319336.060 5006173.580, -831...",None,None,None,22666.4431,Upper Musconetcong,01060000206A7F000001000000010300000002000000F6...


# Task 3d. Aggregate Pollution Loads by Geography

**Stage 1 Task 3.d. Aggregate pollution estimates...**
- ... from all hotspots to produce subbasin (HUC 12) and DRWI cluster-specific estimates of: 
  - 1) total NPS average annual loads, 
  - 2) non-point NPS average annual loads and 
  - 3) point-source NPS average annual loads

Tasks 3c & 3d form **DRWI Pollution Assement Goal 2a**:
- Goal 2. Quantify progress toward improving water quality by DRWI-style land protection and restoration activities.
  - a. What fraction of excess nonpoint source pollution has already been reduced?
  - b. What level of investment is still required to achieve acceptable water quality?

## Excess & Remaining Loads

In [171]:
# Back calculate Loads (kg/y) from excess & remaining loading rates (kg/ha/y)
for pollutant in ['tn', 'tp', 'tss']:
    # Excess NPS Loads (_xsnps)
    if pollutant != 'tss':
        base_catch_gdf[f'{pollutant}_load_xsnps'] = base_catch_gdf[f'{pollutant}_loadrate_xsnps'] \
                                                      * base_catch_gdf.catchment_hectares
    else:  # TSS has no point sources, but let's label it as _xsnps for consistency
        base_catch_gdf[f'{pollutant}_load_xsnps'] = base_catch_gdf[f'{pollutant}_loadrate_xs'] \
                                                      * base_catch_gdf.catchment_hectares

    # Remaining Loads (_rem)
    base_catch_gdf[f'{pollutant}_load_rem'] = base_catch_gdf[f'{pollutant}_loadrate_rem'] \
                                                * base_catch_gdf.catchment_hectares

In [172]:
base_catch_gdf.head(3)

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,...,tss_loadrate_rem,tn_loadrate_rem_p,tp_loadrate_rem_p,tss_loadrate_rem_p,tn_load_xsnps,tn_load_rem,tp_load_xsnps,tp_load_rem,tss_load_xsnps,tss_load_rem
comid,,,,,,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,"MULTIPOLYGON (((-8301340.781 5199034.787, -830...",...,-863.702900,0.0,0.0,0.0,-100576.327864,-100576.327864,-1132.516812,-1132.516812,-5.611223e+06,-5.611223e+06
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,"MULTIPOLYGON (((-8304909.314 5200051.727, -830...",...,-870.834472,0.0,0.0,0.0,-25224.724284,-25224.724284,-218.947572,-218.947572,-1.448347e+06,-1.448347e+06
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,"MULTIPOLYGON (((-8315191.630 5191704.467, -831...",...,-852.303354,0.0,0.0,0.0,-25168.350796,-25168.350796,-157.296268,-157.296268,-1.397277e+06,-1.397277e+06


### Add HUC8 and DRB tags to each COMID

In [173]:
# temp_df = pd.read_excel(data_folder /'comids0204_in_drb.xlsx')
temp_df = pd.read_csv(data_folder /'comids0204_in_drb.csv',
                      index_col='comid',
                      dtype={'in_drb': 'category',
                             'huc08': 'category',
                            }
                     )
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21837 entries, 6261932 to 8409259
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   in_drb  21837 non-null  category
 1   huc08   21837 non-null  category
dtypes: category(2)
memory usage: 213.9 KB


In [174]:
%%time
temp_df.in_drb.value_counts()

Wall time: 2.93 ms


t    16033
f     5804
Name: in_drb, dtype: int64

In [175]:
# Add columns to df
base_catch_gdf[['in_drb','huc08']] = temp_df[['in_drb','huc08']]

In [176]:
base_catch_gdf.in_drb.value_counts()

t    16033
f     3463
Name: in_drb, dtype: int64

## Sum all DRWI

In [177]:
drwi_load_df = base_catch_gdf.loc[:,['catchment_hectares',        # catcment area
                                    'cluster','phase','fa_name','huc12','huc08', # cluster, fa, huc12 categories. Will be dropped if not used for grouping, because not summable.
                                    'tp_load','tn_load','tss_load',    # baseline loads
                                    'tn_load_ps','tp_load_ps',    # point source loads
                                    'tn_load_avoid','tp_load_avoid','tss_load_avoid', # avoided loads from land protection
                                    'tn_load_xsnps','tp_load_xsnps','tss_load_xsnps', # excess nonpoint source loads
                                    'tn_load_rem','tp_load_rem','tss_load_rem', # remaining loads after restoration
                                    ]
                                 ].sum()

drwi_load_df

C:\Users\sjordan\AppData\Local\Temp/ipykernel_17768/1611017827.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  drwi_load_df = base_catch_gdf.loc[:,['catchment_hectares',        # catcment area


catchment_hectares    3.786557e+06
tp_load               3.928625e+06
tn_load               4.800792e+07
tss_load              1.881303e+09
tn_load_ps            2.585555e+07
tp_load_ps            2.498105e+06
tn_load_avoid         4.831585e+04
tp_load_avoid         1.194152e+04
tss_load_avoid        3.238572e+07
tn_load_xsnps        -4.248415e+07
tp_load_xsnps         2.566874e+05
tss_load_xsnps       -1.616718e+09
tn_load_rem          -4.253843e+07
tp_load_rem           2.405809e+05
tss_load_rem         -1.629313e+09
dtype: float64

In [178]:
# Save to CSV file
drwi_load_df.to_csv(data_folder /'drwi_load_all.csv')

### DRWI Loads not in Clusters

In [179]:
base_catch_gdf['cluster'].value_counts(dropna=False)

drb                               8536
Kirkwood - Cohansey Aquifer       3224
NaN                               2138
Poconos and Kittatinny            2069
Upper Lehigh                       962
New Jersey Highlands               795
Brandywine and Christina           767
Middle Schuylkill                  717
Schuylkill Highlands               187
Upstream Suburban Philadelphia     101
Name: cluster, dtype: int64

In [180]:
# Develop mask
mask = base_catch_gdf['cluster'].isnull()
mask.value_counts()

False    17358
True      2138
Name: cluster, dtype: int64

In [181]:
# Sum loads for DRWI, excluding Clusters via mask
mask = base_catch_gdf['cluster'].isnull()

# Preselect colums to keep
# Non-summable dtypes (object, category, geometry) will be dropped automatically
drwi_load_noClus_df = base_catch_gdf[mask].loc[:,['catchment_hectares',        # catcment area
                                                'cluster','phase','fa_name','huc12','huc08', # cluster, fa, huc12 categories. Will be dropped if not used for grouping, because not summable.
                                                'tp_load','tn_load','tss_load',    # baseline loads
                                                'tn_load_ps','tp_load_ps',    # point source loads
                                                'tn_load_avoid','tp_load_avoid','tss_load_avoid', # avoided loads from land protection
                                                'tn_load_xsnps','tp_load_xsnps','tss_load_xsnps', # excess nonpoint source loads
                                                'tn_load_rem','tp_load_rem','tss_load_rem', # remaining loads after restoration
                                                ]
                                            ].sum()


drwi_load_noClus_df

C:\Users\sjordan\AppData\Local\Temp/ipykernel_17768/1580444091.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  drwi_load_noClus_df = base_catch_gdf[mask].loc[:,['catchment_hectares',        # catcment area


catchment_hectares    1.403987e+05
tp_load               5.287418e+04
tn_load               7.677717e+05
tss_load              1.137583e+08
tn_load_ps            0.000000e+00
tp_load_ps            0.000000e+00
tn_load_avoid         0.000000e+00
tp_load_avoid         0.000000e+00
tss_load_avoid        0.000000e+00
tn_load_xsnps        -1.628834e+06
tp_load_xsnps         9.350587e+03
tss_load_xsnps       -1.594200e+07
tn_load_rem          -1.628834e+06
tp_load_rem           9.350587e+03
tss_load_rem         -1.594200e+07
dtype: float64

In [182]:
# Save to CSV file
drwi_load_noClus_df.to_csv(data_folder /'drwi_load_noClus.csv')

### DRWI loads in DRB

In [183]:
# Develop mask
mask = base_catch_gdf['in_drb'] == 't'
mask.value_counts()

True     16033
False     3463
Name: in_drb, dtype: int64

In [184]:
# Sum loads for DRB, excluding Clusters via mask
mask = base_catch_gdf['in_drb'] == 't'

# Preselect colums to keep
# Non-summable dtypes (object, category, geometry) will be dropped automatically
drwi_load_drb_df = base_catch_gdf[mask].loc[:,['catchment_hectares',        # catcment area
                                                'cluster','phase','fa_name','huc12','huc08', # cluster, fa, huc12 categories. Will be dropped if not used for grouping, because not summable.
                                                'tp_load','tn_load','tss_load',    # baseline loads
                                                'tn_load_ps','tp_load_ps',    # point source loads
                                                'tn_load_avoid','tp_load_avoid','tss_load_avoid', # avoided loads from land protection
                                                'tn_load_xsnps','tp_load_xsnps','tss_load_xsnps', # excess nonpoint source loads
                                                'tn_load_rem','tp_load_rem','tss_load_rem', # remaining loads after restoration
                                                ]
                                            ].sum()

C:\Users\sjordan\AppData\Local\Temp/ipykernel_17768/3608248024.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  drwi_load_drb_df = base_catch_gdf[mask].loc[:,['catchment_hectares',        # catcment area


In [185]:
drwi_load_drb_df

catchment_hectares    3.368532e+06
tp_load               3.842764e+06
tn_load               4.652003e+07
tss_load              1.699502e+09
tn_load_ps            2.585555e+07
tp_load_ps            2.498105e+06
tn_load_avoid         4.361818e+04
tp_load_avoid         1.108740e+04
tss_load_avoid        2.846507e+07
tn_load_xsnps        -3.683635e+07
tp_load_xsnps         3.004144e+05
tss_load_xsnps       -1.412347e+09
tn_load_rem          -3.689063e+07
tp_load_rem           2.843079e+05
tss_load_rem         -1.424942e+09
dtype: float64

In [186]:
# Save to CSV file
drwi_load_drb_df.to_csv(data_folder /'drwi_load_drb.csv')

## Sum by Cluster

In [187]:
base_catch_gdf.cluster.value_counts()

drb                               8536
Kirkwood - Cohansey Aquifer       3224
Poconos and Kittatinny            2069
Upper Lehigh                       962
New Jersey Highlands               795
Brandywine and Christina           767
Middle Schuylkill                  717
Schuylkill Highlands               187
Upstream Suburban Philadelphia     101
Name: cluster, dtype: int64

In [188]:
base_catch_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

In [189]:
# Sum loads by Cluster categories
# Preselect colums to keep
# Non-summable dtypes (object, category, geometry) will be dropped automatically

cluster_load_df = base_catch_gdf.loc[:,['catchment_hectares',        # catcment area
                                        'cluster','phase','fa_name','huc12','huc08', # cluster, fa, huc12 categories. Will be dropped if not used for grouping, because not summable.
                                        'tp_load','tn_load','tss_load',    # baseline loads
                                        'tn_load_ps','tp_load_ps',    # point source loads
                                        'tn_load_avoid','tp_load_avoid','tss_load_avoid', # avoided loads from land protection
                                        'tn_load_xsnps','tp_load_xsnps','tss_load_xsnps', # excess nonpoint source loads
                                        'tn_load_rem','tp_load_rem','tss_load_rem', # remaining loads after restoration
                                       ]
                                    ].groupby('cluster').sum()


cluster_load_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 9 entries, Brandywine and Christina to drb
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   catchment_hectares  9 non-null      float64
 1   tp_load             9 non-null      float64
 2   tn_load             9 non-null      float64
 3   tss_load            9 non-null      float64
 4   tn_load_ps          9 non-null      float64
 5   tp_load_ps          9 non-null      float64
 6   tn_load_avoid       9 non-null      float64
 7   tp_load_avoid       9 non-null      float64
 8   tss_load_avoid      9 non-null      float64
 9   tn_load_xsnps       9 non-null      float64
 10  tp_load_xsnps       9 non-null      float64
 11  tss_load_xsnps      9 non-null      float64
 12  tn_load_rem         9 non-null      float64
 13  tp_load_rem         9 non-null      float64
 14  tss_load_rem        9 non-null      float64
dtypes: float64(15)
memory 

In [190]:
cluster_load_df

,catchment_hectares,tp_load,tn_load,tss_load,tn_load_ps,tp_load_ps,tn_load_avoid,tp_load_avoid,tss_load_avoid,tn_load_xsnps,tp_load_xsnps,tss_load_xsnps,tn_load_rem,tp_load_rem,tss_load_rem
cluster,,,,,,,,,,,,,,,
Brandywine and Christina,1.457391e+05,1.179399e+05,2.032147e+06,1.035335e+08,3.738988e+05,3.427440e+04,0.000000,0.000000,0.000000e+00,-8.295176e+05,38486.410864,-3.110023e+07,-8.391640e+05,34905.438964,-3.620600e+07
Kirkwood - Cohansey Aquifer,5.501796e+05,1.608202e+05,2.849539e+06,1.821638e+08,1.112279e+06,1.042147e+05,12329.893071,2019.534175,8.321023e+06,-7.654306e+06,-113950.130138,-3.260922e+08,-7.663279e+06,-115288.342638,-3.279274e+08
Middle Schuylkill,2.029586e+05,3.000070e+05,3.441219e+06,1.409248e+08,1.614086e+06,1.463619e+05,0.000000,0.000000,0.000000e+00,-1.637369e+06,90727.984536,-4.656835e+07,-1.653036e+06,85681.974636,-4.780632e+07
New Jersey Highlands,1.786471e+05,1.625574e+05,1.640035e+06,8.774877e+07,4.372969e+05,6.780901e+04,4919.815508,1274.523078,3.472097e+06,-1.846769e+06,39367.732181,-7.728546e+07,-1.864702e+06,33964.732881,-8.074410e+07
Poconos and Kittatinny,3.424621e+05,6.898337e+04,8.766109e+05,7.115133e+07,8.815656e+04,1.489326e+04,20228.585039,5780.537560,1.440677e+07,-5.057374e+06,-52073.156325,-2.452152e+08,-5.057374e+06,-52073.156325,-2.452152e+08
Schuylkill Highlands,4.485511e+04,5.438029e+04,6.690553e+05,2.801976e+07,1.730334e+05,2.294281e+04,974.867179,237.487951,5.932880e+05,-2.696549e+05,17532.393629,-1.341739e+07,-2.698839e+05,17446.528029,-1.344141e+07
Upper Lehigh,1.980298e+05,5.995963e+04,5.344616e+05,5.447931e+07,9.058475e+04,1.624601e+04,7736.800870,2162.858392,4.620667e+06,-2.936491e+06,-17675.608638,-1.284606e+08,-2.936491e+06,-17675.608638,-1.284606e+08
Upstream Suburban Philadelphia,3.741109e+04,5.937183e+04,3.720539e+05,4.108662e+07,6.441039e+04,4.278428e+04,0.000000,0.000000,0.000000e+00,-3.309639e+05,4990.105755,6.526249e+06,-3.322064e+05,4605.097055,5.737168e+06
drb,1.945876e+06,2.891731e+06,3.482503e+07,1.058437e+09,2.190180e+07,2.048578e+06,2125.890515,466.575131,9.718786e+05,-2.029287e+07,239931.048979,-7.391631e+08,-2.029346e+07,239663.623379,-7.393067e+08


In [191]:
# Save to CSV file
cluster_load_df.to_csv(data_folder /'cluster_loads.csv')

In [192]:
# merge or join with this?
cluster_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 8 entries, Poconos and Kittatinny to Upper Lehigh
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   gid         8 non-null      int64   
 1   objectid    8 non-null      int64   
 2   label_id    8 non-null      int64   
 3   sum_acres   8 non-null      float64 
 4   acres       8 non-null      float64 
 5   ncontrb_a   8 non-null      float64 
 6   acres_ans   8 non-null      int64   
 7   name        8 non-null      category
 8   shape_leng  8 non-null      float64 
 9   shape_area  8 non-null      float64 
 10  geom        8 non-null      geometry
dtypes: category(1), float64(5), geometry(1), int64(4)
memory usage: 1.4 KB


## Sum by Focus Area within Clusters

In [193]:
base_catch_gdf.phase.value_counts()

Phase 1    2708
Phase 2    1374
Name: phase, dtype: int64

In [194]:
# Create merged name for Focus Area by Phase
base_catch_gdf['fa_name_phase'] = base_catch_gdf.phase.dropna().astype('str') + \
                                  ' ' + base_catch_gdf.fa_name.dropna().astype('str')

base_catch_gdf['fa_name_phase'] = base_catch_gdf['fa_name_phase'].astype('category')

In [195]:
base_catch_gdf.fa_name_phase.value_counts()

Phase 1 Core Pine Barrens                918
Phase 1 Cohansey-Maurice                 399
Phase 1 Salem River                      279
Phase 1 Bush Kill/Hornbecks Creek        254
Phase 1 Upper Delaware River Corridor    193
                                        ... 
Phase 2 Lower Maiden Cr Trib 2             1
Phase 2 Upper French Creek                 1
Phase 2 Pine Creek/Pickering Creek         1
Phase 2 Lower Maiden Cr Trib 3             1
Phase 2 Sixpenny                           1
Name: fa_name_phase, Length: 97, dtype: int64

In [196]:
# Sum loads by Focus Area categories
# Preselect colums to keep
# Non-summable dtypes (object, category, geometry) will be dropped automatically

focusarea_load_df = base_catch_gdf.loc[:,['catchment_hectares',        # catcment area
                                          'cluster','phase','fa_name','fa_name_phase',
                                          'huc12','huc08', # cluster, fa, huc12 categories. Will be dropped if not used for grouping, because not summable.
                                          'tp_load','tn_load','tss_load',    # baseline loads
                                          'tn_load_ps','tp_load_ps',    # point source loads
                                          'tn_load_avoid','tp_load_avoid','tss_load_avoid', # avoided loads from land protection
                                          'tn_load_xsnps','tp_load_xsnps','tss_load_xsnps', # excess nonpoint source loads
                                          'tn_load_rem','tp_load_rem','tss_load_rem', # remaining loads after restoration
                                         ]
                                       ].groupby('fa_name_phase').sum()
                                    


focusarea_load_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 97 entries, Phase 1 Bear Creek to Phase 2 White Clay Creek
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   catchment_hectares  97 non-null     float64
 1   tp_load             97 non-null     float64
 2   tn_load             97 non-null     float64
 3   tss_load            97 non-null     float64
 4   tn_load_ps          97 non-null     float64
 5   tp_load_ps          97 non-null     float64
 6   tn_load_avoid       97 non-null     float64
 7   tp_load_avoid       97 non-null     float64
 8   tss_load_avoid      97 non-null     float64
 9   tn_load_xsnps       97 non-null     float64
 10  tp_load_xsnps       97 non-null     float64
 11  tss_load_xsnps      97 non-null     float64
 12  tn_load_rem         97 non-null     float64
 13  tp_load_rem         97 non-null     float64
 14  tss_load_rem        97 non-null     float64
dtypes: flo

In [197]:
focusarea_load_df

,catchment_hectares,tp_load,tn_load,tss_load,tn_load_ps,tp_load_ps,tn_load_avoid,tp_load_avoid,tss_load_avoid,tn_load_xsnps,tp_load_xsnps,tss_load_xsnps,tn_load_rem,tp_load_rem,tss_load_rem
fa_name_phase,,,,,,,,,,,,,,,
Phase 1 Bear Creek,7167.2606,1461.6925,16744.9751,1.909422e+06,0.000000,0.000000,458.105988,177.465310,4.063045e+05,-1.056002e+05,-760.158286,-4.711693e+06,-1.056002e+05,-760.158286,-4.711693e+06
Phase 1 Bush Kill/Hornbecks Creek,44484.9440,7339.1062,72332.8907,7.989250e+06,1018.487958,2270.555448,3790.305768,1104.014885,3.001218e+06,-6.880436e+05,-8721.781888,-3.310594e+07,-6.880436e+05,-8721.781888,-3.310594e+07
Phase 1 Cohansey-Maurice,79845.4069,34983.5087,739913.7360,2.890817e+07,485963.035662,43386.253012,0.000000,0.000000,0.000000e+00,-1.109010e+06,-33154.820451,-4.485302e+07,-1.109440e+06,-33199.203151,-4.490463e+07
Phase 1 Core Pine Barrens,131694.8683,12435.4862,194341.9482,2.443837e+07,4984.340007,427.229143,5802.761775,1032.918245,4.577838e+06,-2.058674e+06,-28817.152116,-9.722135e+07,-2.058674e+06,-28817.152116,-9.722135e+07
Phase 1 French Creek Headwaters,4599.8253,3123.7215,53199.0641,2.511461e+06,1.949996,38.159923,237.597794,59.948665,1.563532e+05,-2.532190e+04,1659.615734,-1.737858e+06,-2.532190e+04,1659.615734,-1.737858e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Phase 2 Upper Musconetcong,9168.7891,1979.4624,34800.6153,3.680347e+06,0.000000,0.000000,4065.155014,1066.078594,2.941400e+06,-1.217106e+05,-862.862221,-4.789781e+06,-1.217106e+05,-862.862221,-4.789781e+06
Phase 2 Upper Neversink,8649.9809,872.6068,22613.2745,7.439836e+05,556.968883,69.039862,0.000000,0.000000,0.000000e+00,-1.255989e+05,-1877.927141,-7.246869e+06,-1.255989e+05,-1877.927141,-7.246869e+06
Phase 2 Upper Salem River,8402.0393,6666.1852,148177.3137,5.511011e+06,9686.210579,98.619802,0.000000,0.000000,0.000000e+00,-4.931708e+03,3962.933215,-2.250792e+06,-5.181585e+03,3839.339615,-2.344295e+06


### Add `cluster`, `huc12`, `geom` from `focusarea_gdf`

In [198]:
base_catch_gdf.loc[:,['fa_name_phase','cluster', 'phase','fa_name']].dropna().drop_duplicates()

,fa_name_phase,cluster,phase,fa_name
comid,,,,
2583185,Phase 1 Upper Paulins Kill,New Jersey Highlands,Phase 1,Upper Paulins Kill
2583187,Phase 2 Paulinskill,New Jersey Highlands,Phase 2,Paulinskill
2583487,Phase 1 Upper Musconetcong,New Jersey Highlands,Phase 1,Upper Musconetcong
2583493,Phase 2 Upper Musconetcong,New Jersey Highlands,Phase 2,Upper Musconetcong
2586181,Phase 2 Beaver Brook,New Jersey Highlands,Phase 2,Beaver Brook
...,...,...,...,...
9480512,Phase 2 Upper Salem River,Kirkwood - Cohansey Aquifer,Phase 2,Upper Salem River
9480764,Phase 2 Lower Salem River,Kirkwood - Cohansey Aquifer,Phase 2,Lower Salem River
9480810,Phase 2 Muddy Run,drb,Phase 2,Muddy Run


In [199]:
# Add back categoricals that were dropped
left = focusarea_load_df.copy()
right = base_catch_gdf.loc[:,['fa_name_phase','cluster', 'phase','fa_name']].dropna().drop_duplicates()
right

,fa_name_phase,cluster,phase,fa_name
comid,,,,
2583185,Phase 1 Upper Paulins Kill,New Jersey Highlands,Phase 1,Upper Paulins Kill
2583187,Phase 2 Paulinskill,New Jersey Highlands,Phase 2,Paulinskill
2583487,Phase 1 Upper Musconetcong,New Jersey Highlands,Phase 1,Upper Musconetcong
2583493,Phase 2 Upper Musconetcong,New Jersey Highlands,Phase 2,Upper Musconetcong
2586181,Phase 2 Beaver Brook,New Jersey Highlands,Phase 2,Beaver Brook
...,...,...,...,...
9480512,Phase 2 Upper Salem River,Kirkwood - Cohansey Aquifer,Phase 2,Upper Salem River
9480764,Phase 2 Lower Salem River,Kirkwood - Cohansey Aquifer,Phase 2,Lower Salem River
9480810,Phase 2 Muddy Run,drb,Phase 2,Muddy Run


In [200]:

x = pd.merge(left,right, on='fa_name_phase').set_index('fa_name_phase')
x

,catchment_hectares,tp_load,tn_load,tss_load,tn_load_ps,tp_load_ps,tn_load_avoid,tp_load_avoid,tss_load_avoid,tn_load_xsnps,tp_load_xsnps,tss_load_xsnps,tn_load_rem,tp_load_rem,tss_load_rem,cluster,phase,fa_name
fa_name_phase,,,,,,,,,,,,,,,,,,
Phase 1 Bear Creek,7167.2606,1461.6925,16744.9751,1.909422e+06,0.000000,0.000000,458.105988,177.465310,4.063045e+05,-1.056002e+05,-760.158286,-4.711693e+06,-1.056002e+05,-760.158286,-4.711693e+06,Upper Lehigh,Phase 1,Bear Creek
Phase 1 Bush Kill/Hornbecks Creek,44484.9440,7339.1062,72332.8907,7.989250e+06,1018.487958,2270.555448,3790.305768,1104.014885,3.001218e+06,-6.880436e+05,-8721.781888,-3.310594e+07,-6.880436e+05,-8721.781888,-3.310594e+07,Poconos and Kittatinny,Phase 1,Bush Kill/Hornbecks Creek
Phase 1 Cohansey-Maurice,79845.4069,34983.5087,739913.7360,2.890817e+07,485963.035662,43386.253012,0.000000,0.000000,0.000000e+00,-1.109010e+06,-33154.820451,-4.485302e+07,-1.109440e+06,-33199.203151,-4.490463e+07,Kirkwood - Cohansey Aquifer,Phase 1,Cohansey-Maurice
Phase 1 Core Pine Barrens,131694.8683,12435.4862,194341.9482,2.443837e+07,4984.340007,427.229143,5802.761775,1032.918245,4.577838e+06,-2.058674e+06,-28817.152116,-9.722135e+07,-2.058674e+06,-28817.152116,-9.722135e+07,Kirkwood - Cohansey Aquifer,Phase 1,Core Pine Barrens
Phase 1 French Creek Headwaters,4599.8253,3123.7215,53199.0641,2.511461e+06,1.949996,38.159923,237.597794,59.948665,1.563532e+05,-2.532190e+04,1659.615734,-1.737858e+06,-2.532190e+04,1659.615734,-1.737858e+06,Schuylkill Highlands,Phase 1,French Creek Headwaters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Phase 2 Upper Musconetcong,9168.7891,1979.4624,34800.6153,3.680347e+06,0.000000,0.000000,4065.155014,1066.078594,2.941400e+06,-1.217106e+05,-862.862221,-4.789781e+06,-1.217106e+05,-862.862221,-4.789781e+06,New Jersey Highlands,Phase 2,Upper Musconetcong
Phase 2 Upper Neversink,8649.9809,872.6068,22613.2745,7.439836e+05,556.968883,69.039862,0.000000,0.000000,0.000000e+00,-1.255989e+05,-1877.927141,-7.246869e+06,-1.255989e+05,-1877.927141,-7.246869e+06,Poconos and Kittatinny,Phase 2,Upper Neversink
Phase 2 Upper Salem River,8402.0393,6666.1852,148177.3137,5.511011e+06,9686.210579,98.619802,0.000000,0.000000,0.000000e+00,-4.931708e+03,3962.933215,-2.250792e+06,-5.181585e+03,3839.339615,-2.344295e+06,Kirkwood - Cohansey Aquifer,Phase 2,Upper Salem River


In [201]:
focusarea_load_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 97 entries, Phase 1 Bear Creek to Phase 2 White Clay Creek
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   catchment_hectares  97 non-null     float64
 1   tp_load             97 non-null     float64
 2   tn_load             97 non-null     float64
 3   tss_load            97 non-null     float64
 4   tn_load_ps          97 non-null     float64
 5   tp_load_ps          97 non-null     float64
 6   tn_load_avoid       97 non-null     float64
 7   tp_load_avoid       97 non-null     float64
 8   tss_load_avoid      97 non-null     float64
 9   tn_load_xsnps       97 non-null     float64
 10  tp_load_xsnps       97 non-null     float64
 11  tss_load_xsnps      97 non-null     float64
 12  tn_load_rem         97 non-null     float64
 13  tp_load_rem         97 non-null     float64
 14  tss_load_rem        97 non-null     float64
dtypes: flo

In [202]:
# Save to CSV file, excluding geometries.
focusarea_load_df.to_csv(data_folder /'focusarea_loads.csv')
x.sort_values('cluster').to_csv(data_folder /'focusarea_loads_byCluster.csv')

### Cluster loads NOT IN Focus Area (noFA)

In [203]:
base_catch_gdf['fa_name_phase'].value_counts(dropna=False)

NaN                                  15414
Phase 1 Core Pine Barrens              918
Phase 1 Cohansey-Maurice               399
Phase 1 Salem River                    279
Phase 1 Bush Kill/Hornbecks Creek      254
                                     ...  
Phase 2 Lower Maiden Cr Trib 3           1
Phase 2 Sixpenny                         1
Phase 1 Stony Creek                      1
Phase 2 Saucony Cr Trib 1                1
Phase 2 Lower Maiden Cr Trib 2           1
Name: fa_name_phase, Length: 98, dtype: int64

In [204]:
# Develop mask
mask = base_catch_gdf['fa_name_phase'].isnull()
mask.value_counts()

True     15414
False     4082
Name: fa_name_phase, dtype: int64

In [205]:
# Sum loads by Cluster categories, excluding Focus Areas via mask
mask = base_catch_gdf['fa_name_phase'].isnull()

# Preselect colums to keep
# Non-summable dtypes (object, category, geometry) will be dropped automatically
cluster_load_noFA_df = base_catch_gdf[mask].loc[:,['catchment_hectares',        # catcment area
                                                    'cluster','phase','fa_name','fa_name_phase',
                                                    'huc12','huc08', # cluster, fa, huc12 categories. Will be dropped if not used for grouping, because not summable.
                                                    'tp_load','tn_load','tss_load',    # baseline loads
                                                    'tn_load_ps','tp_load_ps',    # point source loads
                                                    'tn_load_avoid','tp_load_avoid','tss_load_avoid', # avoided loads from land protection
                                                    'tn_load_xsnps','tp_load_xsnps','tss_load_xsnps', # excess nonpoint source loads
                                                    'tn_load_rem','tp_load_rem','tss_load_rem', # remaining loads after restoration
                                                    ]
                                                ].groupby('cluster').sum()

cluster_load_noFA_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 9 entries, Brandywine and Christina to drb
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   catchment_hectares  9 non-null      float64
 1   tp_load             9 non-null      float64
 2   tn_load             9 non-null      float64
 3   tss_load            9 non-null      float64
 4   tn_load_ps          9 non-null      float64
 5   tp_load_ps          9 non-null      float64
 6   tn_load_avoid       9 non-null      float64
 7   tp_load_avoid       9 non-null      float64
 8   tss_load_avoid      9 non-null      float64
 9   tn_load_xsnps       9 non-null      float64
 10  tp_load_xsnps       9 non-null      float64
 11  tss_load_xsnps      9 non-null      float64
 12  tn_load_rem         9 non-null      float64
 13  tp_load_rem         9 non-null      float64
 14  tss_load_rem        9 non-null      float64
dtypes: float64(15)
memory 

In [206]:
cluster_load_noFA_df

,catchment_hectares,tp_load,tn_load,tss_load,tn_load_ps,tp_load_ps,tn_load_avoid,tp_load_avoid,tss_load_avoid,tn_load_xsnps,tp_load_xsnps,tss_load_xsnps,tn_load_rem,tp_load_rem,tss_load_rem
cluster,,,,,,,,,,,,,,,
Brandywine and Christina,1.268644e+05,9.731760e+04,1.683407e+06,9.019894e+07,3.652659e+05,3.346987e+04,0.000000,0.000000,0.000000e+00,-8.474341e+05,24519.756042,-2.699840e+07,-8.493342e+05,23939.518442,-2.753866e+07
Kirkwood - Cohansey Aquifer,2.293606e+05,5.124876e+04,9.393489e+05,7.432373e+07,1.902079e+05,2.014795e+04,518.853819,82.249098,3.224214e+05,-3.166045e+06,-40000.986807,-1.375596e+08,-3.166045e+06,-40000.986807,-1.375596e+08
Middle Schuylkill,1.721850e+05,2.508254e+05,2.903346e+06,1.202878e+08,1.609896e+06,1.438194e+05,0.000000,0.000000,0.000000e+00,-1.645748e+06,53628.716955,-3.877669e+07,-1.651951e+06,51863.755555,-3.958269e+07
New Jersey Highlands,9.795376e+04,7.120166e+04,1.026635e+06,4.516842e+07,2.055212e+05,1.488968e+04,86.957789,20.677442,5.277441e+04,-8.509566e+05,25946.311640,-4.532126e+07,-8.516394e+05,25593.414540,-4.547667e+07
Poconos and Kittatinny,1.826341e+05,4.308559e+04,4.867389e+05,4.120057e+07,7.449627e+04,1.137804e+04,3935.564572,1119.157569,2.774932e+06,-2.705321e+06,-24909.005637,-1.275168e+08,-2.705321e+06,-24909.005637,-1.275168e+08
Schuylkill Highlands,2.056135e+04,3.589053e+04,4.002707e+05,1.512012e+07,1.729944e+05,2.286406e+04,77.410388,19.238270,5.021138e+04,-1.237058e+05,6652.452130,-3.874447e+06,-1.237063e+05,6652.189730,-3.874447e+06
Upper Lehigh,1.427041e+05,5.082162e+04,4.367324e+05,4.350306e+07,8.007847e+04,1.529990e+04,437.272748,160.865520,3.686535e+05,-2.079305e+06,-8716.549258,-8.832698e+07,-2.079305e+06,-8716.549258,-8.832698e+07
Upstream Suburban Philadelphia,2.684584e+04,4.906710e+04,2.027527e+05,2.974475e+07,1.427634e+04,3.825994e+04,0.000000,0.000000,0.000000e+00,-2.697822e+05,2484.943900,4.944559e+06,-2.698990e+05,2452.448300,4.920205e+06
drb,1.944356e+06,2.890822e+06,3.480761e+07,1.057739e+09,2.190169e+07,2.048569e+06,2125.890515,466.575131,9.718786e+05,-2.028424e+07,239502.491441,-7.384574e+08,-2.028482e+07,239238.944841,-7.385973e+08


In [207]:
# Save to CSV file
cluster_load_noFA_df.to_csv(data_folder /'cluster_load_noFA.csv')

To visualize the data analyzed in this notebook, refer to WikiSRAT_Viz.ipynd